# 这个笔记本来自:,如果你有什么问题,可以来提交问题

In [ ]:
#@markdown # 挂载谷歌的云盘
#@markdown 这个步骤相当于把谷歌的网盘当作硬盘来使用,以后就不需要每次都重新下载环境,下载模型等操作了.


from google.colab import drive
drive.mount('/content/gdrive')
!nvidia-smi

## 初始化环境

如果你是第一次运行,请运行此步骤.



In [ ]:
#@markdown # 安装/更新 Stable Diffusion
#@markdown 更新也可以单独运行这个节点

from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  raise RuntimeError('[1;31mGoogle云盘未挂载!请先挂载网盘.')

with capture.capture_output() as cap:
  def inf(msg, style, wdth): 
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)
  
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf A1111_dep.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

clear_output()
inf('\u2714 完成','success', '50px')

In [ ]:
#@markdown # 下载模型

link = "" #@param {type:'string'}
name = "" #@param {type:'string'}
path = 'CheckPoint' #@param ["CheckPoint", "Lora", "VAE"]
safetensors = True #@param {type:"boolean"}

#@markdown 参数说明
#@markdown - link: 模型的下载地址.
#@markdown - name: 存储到本地的模型名称,为空则为默认
#@markdown - Path: 选择相应的存储目录.
#@markdown - safetensors: 是否下载safetensors格式的模型,推荐选中.


if path == 'CheckPoint': path = 'Stable-diffusion'
#google目录
gd_path = f'/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/{path}'

%cd $gd_path
clear_output()

if not link:
  raise ValueError('请输入下载链接')


if not safetensors:
    modelname=f"{name}.ckpt"
else:
  modelname=f"{name}.safetensors"

#开始下载
!gdown --fuzzy -O $modelname $link


inf('\u2714 下载完成.','success', '350px')




In [ ]:
#@markdown # 安装ControlNet以及模型
from torch.hub import download_url_to_file
from urllib.parse import urlparse

Model = "Canny" #@param [ "All (5.6GB)", "Canny", "Depth", "HED", "MLSD", "Normal", "OpenPose", "Scribble", "Seg"]

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
Depth='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
HED='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
MLSD='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
Normal='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
OpenPose='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
Scribble='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
Seg='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'

all_cn = [Canny,Depth,HED,MLSD,Normal,OpenPose,Scribble,Seg]

# clone or pull ControlNet
with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git pull
    %cd /content

!cp /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/*.yaml /content/gdrive/MyDrive/sd/stable-diffusion-webui/models
cndir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models"


if Model == 'All (5.6GB)':
  for lnk in all_cn:
    download(url=lnk, model_dir=cndir)
  clear_output()
  inf('\u2714 完成','success', '50px')

else:
  download(globals()[Model], model_dir=cndir)
  clear_output()
  inf('\u2714 完成','success', '50px')



In [ ]:
#@markdown # 安装Mov2mov以及模型

modnet_webcam = '1Nf1ZxeJZJL8Qx9KadcYYyEmmlKhTADxX'
modnet_photographic = '1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz'

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-mov2mov"):
    !git clone https://github.com/Scholar01/sd-webui-mov2mov.git
    %cd /content
  else:
    %cd sd-webui-mov2mov
    !git pull
    %cd /content

all_cn = [modnet_webcam,modnet_photographic]

cndir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-mov2mov/models"
%cd $cndir
for lnk in all_cn:
  !gdown --fuzzy $lnk

clear_output() 
inf('\u2714 完成','success', '50px')


## 一键启动

- 一键启动并配置Mov2mov运行环境

In [ ]:
from IPython.utils import capture
import os
import time
import sys
import fileinput
from subprocess import getoutput

Use_localtunnel = True #@param {type:'boolean'} 
User = ''#@param{type:'string'}
Password = ''#@param{type:'string'}

#@markdown 参数说明
#@markdown - Use_localtunnel: 启动本地ssh隧道,推荐启用.
#@markdown - User: 设置Webui的登录账号,为空则不设置.(任何人知道你链接都可以连接)
#@markdown - Password: 设置Webui的登录密码,说明同上.


if not os.path.exists("/content/gdrive/MyDrive/"):
  raise RuntimeError('[1;31mGoogle云盘未挂载!请先挂载网盘.')

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel



with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py




share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.8/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars
